<a href="https://colab.research.google.com/github/arina19-2000/unimi/blob/main/tesi/tesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/colab


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/colab


## Data preparation

In [3]:
dfa = pd.read_excel('results adults2.xlsx')

In [4]:
column_types = dfa.dtypes

type_counts = column_types.value_counts()
print("Column types and counts:")
print(type_counts)

Column types and counts:
object     89
float64    20
int64       1
Name: count, dtype: int64


In [5]:
dfa = dfa.dropna(axis=1, how='all')
dfa.shape

(786, 109)

In [6]:
dfa = dfa.dropna(thresh=int(0.05 * dfa.shape[1]), axis=0)
dfa.shape

(495, 109)

In [7]:
dfa = dfa.dropna(thresh=int(0.05 * dfa.shape[1]), axis=1)
dfa.shape

(495, 105)

In [8]:
dfa['Perché ha lasciato?'].head()

,Perché ha lasciato?
0,NaN
2,NaN
3,NaN
4,NaN
5,NaN


In [9]:
dfa.loc[:, 'Qual è il titolo di studio massimo conseguito?'] = dfa['Qual è il titolo di studio massimo conseguito?'].str.extract('(\d)')
dfa.loc[:, 'Quale scuola secondaria di secondo grado (o superiore) ha frequentato?'] = dfa['Quale scuola secondaria di secondo grado (o superiore) ha frequentato?'].str.extract('(\d)')
dfa.loc[:, 'Perché ha lasciato?'] = dfa['Perché ha lasciato?'].str.extract('(\d)')
dfa.loc[:, 'Qual è il titolo di studio massimo conseguito dalla MADRE?'] = dfa['Qual è il titolo di studio massimo conseguito dalla MADRE?'].str.extract('(\d)')
dfa.loc[:, 'Qual è il titolo di studio massimo conseguito dal PADRE?'] = dfa['Qual è il titolo di studio massimo conseguito dal PADRE?'].str.extract('(\d)')

dfa.head()

,ID risposta,Ha uno o più figli adottati nati prima del 2002 (escluso)?,Numero di figli adottati nati prima del 2002 (escluso) [N°][],Genere,Anno di nascita [Anno di nascita][],Adozione,Anno di arrivo in famiglia [Anno][],Stato di provenienza,Era scolarizzato nel paese di origine (Italia inclusa)?,Età di inserimento a scuola [Età][],...,Con quale valutazione? [Voto][.../60].3,Ha conseguito una laurea triennale o magistrale?.3,Voto della laurea? [Voto][.../110],Professione del PADRE quando il figlio ha fatto la scelta della scuola secondaria di II° grado o superiore:,Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [No],"Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alle primarie]","Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alla secondaria di primo grado]","Il PADRE è mai stato negli organi collegiali delle scuole dei propri figli? [Sì, alla secondaria di secondo grado]",Il PADRE ha dovuto ripetere degli anni alla scuola secondaria di secondo grado (o superiori)?,Il PADRE ha avuto difficoltà di rendimento/apprendimento che lo hanno spinto a cambiare tipologia di indirizzo di scuola secondaria di secondo grado (o superiore)?
0,1,Sì,1.0,Femmina,1999.0,Nazionale,2004.0,Italia,No,5.0,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,programmista,No,No,No,Sì,"Sì, 1 anno",Sì
2,3,Sì,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,No,6.0,...,NaN,NaN,NaN,Operaio,Sì,No,No,No,NaN,NaN
3,4,Sì,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,No,6.0,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,Impiegato,Sì,No,No,No,No,No
4,5,Sì,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,Sì,4.0,...,NaN,NaN,110.0,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No
5,6,Sì,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,No,6.0,...,60.0,NaN,NaN,Impiegato statale,Sì,No,No,No,No,No


In [10]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('italian'))

def remove_italian_stopwords(col):
    col = col.lower()
    col = re.sub(r'[^\w\s]', '', col)
    # col = re.sub(r'^G\d+\[.*?\]\.\s*', '', col)
    col = re.sub(r'\s+', '_', col)
    words = [word for word in col.split('_') if word not in stop_words]
    return '_'.join(words)

dfa.columns = [remove_italian_stopwords(col) for col in dfa.columns]

dfa.columns


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Index(['id_risposta', 'figli_adottati_nati_prima_2002_escluso',
       'numero_figli_adottati_nati_prima_2002_escluso_n', 'genere',
       'anno_nascita_anno_nascita', 'adozione', 'anno_arrivo_famiglia_anno',
       'stato_provenienza', 'scolarizzato_paese_origine_italia_inclusa',
       'età_inserimento_scuola_età',
       ...
       'valutazione_voto603', 'conseguito_laurea_triennale_magistrale3',
       'voto_laurea_voto110',
       'professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_no',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado',
       'padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado',
       'padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori',
       'padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipolo

In [11]:
dfa = dfa.loc[:, dfa.apply(pd.Series.nunique) > 1]
dfa.shape

(495, 99)

In [12]:
dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,valutazione_voto603,conseguito_laurea_triennale_magistrale3,voto_laurea_voto110,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,Italia,No,5.0,Scuola materna,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,programmista,No,No,No,Sì,"Sì, 1 anno",Sì
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,No,6.0,Prima elementare,...,NaN,NaN,NaN,Operaio,Sì,No,No,No,NaN,NaN
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,No,6.0,Prima elementare,...,NaN,Laurea magistrale o vecchio ordinamento,110.0,Impiegato,Sì,No,No,No,No,No
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,Sì,4.0,Scuola materna,...,NaN,NaN,110.0,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,No,6.0,Scuola materna,...,60.0,NaN,NaN,Impiegato statale,Sì,No,No,No,No,No


In [13]:
dfa.padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie.nunique()

2

## Rename column content

In [14]:
dfa['genere_fem'] = dfa['genere'].apply(lambda x: 1 if x == 'Femmina' else 0)
dfa['genere_fem'] = dfa['genere1'].apply(lambda x: 1 if x == 'Femmina' else 0)
dfa['adozione_int'] = dfa['adozione'].apply(lambda x: 1 if x == 'Internazionale' else 0)
dfa['adozione_int'] = dfa['adozione1'].apply(lambda x: 1 if x == 'Internazionale' else 0)

In [15]:
dfa['adopt_age'] = dfa['anno_arrivo_famiglia_anno'] - dfa['anno_nascita_anno_nascita']


In [16]:
dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,genere_fem,adozione_int,adopt_age
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,Italia,No,5.0,Scuola materna,...,programmista,No,No,No,Sì,"Sì, 1 anno",Sì,0,0,5.0
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,No,6.0,Prima elementare,...,Operaio,Sì,No,No,No,NaN,NaN,1,0,1.0
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,No,6.0,Prima elementare,...,Impiegato,Sì,No,No,No,No,No,0,0,7.0
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,Sì,4.0,Scuola materna,...,Assistente sociale,No,No,No,Sì,"Sì, 1 anno",No,0,0,5.0
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,No,6.0,Scuola materna,...,Impiegato statale,Sì,No,No,No,No,No,0,0,15.0


In [17]:
for col in dfa.columns:
    if isinstance(dfa[col], pd.Series):
        unique_values = dfa[col].dropna().unique()
        if set(unique_values).issubset({'Sì', 'No'}):
            dfa[col] = dfa[col].replace({'Sì': 1, 'No': 0})



dfa.head()

,id_risposta,numero_figli_adottati_nati_prima_2002_escluso_n,genere,anno_nascita_anno_nascita,adozione,anno_arrivo_famiglia_anno,stato_provenienza,scolarizzato_paese_origine_italia_inclusa,età_inserimento_scuola_età,prima_classe_frequenza_scuola,...,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_mai_stato_organi_collegiali_scuole_propri_figli_no,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_primarie,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_primo_grado,padre_mai_stato_organi_collegiali_scuole_propri_figli_sì_secondaria_secondo_grado,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,padre_difficoltà_rendimentoapprendimento_spinto_cambiare_tipologia_indirizzo_scuola_secondaria_secondo_grado_superiore,genere_fem,adozione_int,adopt_age
0,1,1.0,Femmina,1999.0,Nazionale,2004.0,Italia,0.0,5.0,Scuola materna,...,programmista,0.0,0.0,0.0,1.0,"Sì, 1 anno",1.0,0,0,5.0
2,3,2.0,Femmina,1990.0,Nazionale,1991.0,Italia,0.0,6.0,Prima elementare,...,Operaio,1.0,0.0,0.0,0.0,NaN,NaN,1,0,1.0
3,4,1.0,Femmina,1996.0,Internazionale,2003.0,Ucraina,0.0,6.0,Prima elementare,...,Impiegato,1.0,0.0,0.0,0.0,No,0.0,0,0,7.0
4,5,1.0,Femmina,1996.0,Nazionale,2001.0,Italia,1.0,4.0,Scuola materna,...,Assistente sociale,0.0,0.0,0.0,1.0,"Sì, 1 anno",0.0,0,0,5.0
5,6,1.0,Femmina,1999.0,Internazionale,2014.0,Ucraina,0.0,6.0,Scuola materna,...,Impiegato statale,1.0,0.0,0.0,0.0,No,0.0,0,0,15.0


In [18]:
dfa = dfa.drop(columns=['genere','adozione','scuola_secondaria_secondo_grado_superiore_frequentato_altro', 'genere1','adozione1','prima_classe_frequenza_scuola_altro',])

In [19]:
dfa.conseguito_laurea_triennale_magistrale.value_counts()

,count
conseguito_laurea_triennale_magistrale,
Laurea triennale,32
Laurea magistrale,19


In [20]:
hierarchical_mapping = {
    'Asilo nido': 1,
    'Scuola materna': 2,
    'Scuola d\'infanzia': 3,
    'Prima elementare': 4,
    'Seconda elementare': 5,
    'Terza elementare': 6,
    'Quarta elementare': 7,
    'Prima media': 8,
    'Seconda media': 9,
    'Terza media': 10,
    'Altro': 99
}

dfa['prima_classe_frequenza_scuola'] = dfa['prima_classe_frequenza_scuola'].map(hierarchical_mapping)
dfa['prima_classe_frequenza_scuola1'] = dfa['prima_classe_frequenza_scuola1'].map(hierarchical_mapping)


In [21]:
hierarchical_mapping2 = {
    'No': 1,
    'Sì, 1 anno': 1,
    'Sì, 2 anni': 2,
    'Sì, più di 2 anni': 3,

}

dfa['dovuto_ripetere_anni_scuola_secondaria_superiore'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore'].map(hierarchical_mapping2)
dfa['dovuto_ripetere_anni_scuola_secondaria_superiore1'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore1'].map(hierarchical_mapping2)
dfa['dovuto_ripetere_anni_scuola_secondaria_superiore2'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore2'].map(hierarchical_mapping2)
dfa['dovuto_ripetere_anni_scuola_secondaria_superiore3'] = dfa['dovuto_ripetere_anni_scuola_secondaria_superiore3'].map(hierarchical_mapping2)

In [22]:
object_columns = dfa.columns[dfa.dtypes == 'object']
print(object_columns)

Index(['stato_provenienza', 'conseguito_laurea_triennale_magistrale',
       'ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc',
       'lavoro_fa', 'attualmente_frequentando', 'fatto_esperienze_allestero',
       'descrizione_nucleo_familiare_figlio',
       'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_primaria',
       'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_grado',
       'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_ii_grado',
       'secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto

In [23]:
dfa[object_columns].head()

,stato_provenienza,conseguito_laurea_triennale_magistrale,ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc,ambito_titolo_universitario_es_economia_giurispudenza_medicina_etc,lavoro_fa,attualmente_frequentando,fatto_esperienze_allestero,descrizione_nucleo_familiare_figlio,secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_primaria,secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_grado,...,secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_scuola_secondaria_ii_grado1,secondo_esperienza_percorso_scolastico_tuoa_figlioa_rispetto_compagni_classe_adottati_riguarda_rapporto_impegnorisultati_stato_indicare_nessuna_risposta_figlio_frequentato_scuola_indicata_università1,regione_residenza_madre,conseguito_laurea_triennale_magistrale2,professione_madre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,madre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori,regione_residenza_padre,conseguito_laurea_triennale_magistrale3,professione_padre_quando_figlio_fatto_scelta_scuola_secondaria_ii_grado_superiore,padre_dovuto_ripetere_anni_scuola_secondaria_secondo_grado_superiori
0,Italia,NaN,NaN,NaN,NaN,Università,"Sì, di studio (non estivo)",Vive con i genitori,Impegnativo nello stesso modo,Impegnativo nello stesso modo,...,NaN,NaN,Lazio,NaN,Dirigente,"Sì, 2 anni",Lazio,Laurea magistrale o vecchio ordinamento,programmista,"Sì, 1 anno"
2,Italia,Laurea triennale,Lingue e Letterature Straniere,NaN,Back office,NaN,No,Vive con partner e figli,Più impegnativo,Più impegnativo,...,Più impegnativo,NaN,Lombardia,NaN,Impiegata,NaN,Lombardia,NaN,Operaio,NaN
3,Ucraina,NaN,NaN,NaN,NaN,Università,No,Vive con i genitori,Più impegnativo,Più impegnativo,...,NaN,NaN,Sicilia,Laurea magistrale o vecchio ordinamento,Commercialista,No,Sicilia,Laurea magistrale o vecchio ordinamento,Impiegato,No
4,Italia,NaN,NaN,NaN,NaN,NaN,No,Vive con i genitori,Più impegnativo,Più impegnativo,...,NaN,NaN,Sicilia,Laurea triennale,Logopedista,No,Sicilia,NaN,Assistente sociale,"Sì, 1 anno"
5,Ucraina,NaN,NaN,NaN,NaN,NaN,No,Vive con i genitori,Più impegnativo,Più impegnativo,...,NaN,NaN,Puglia,NaN,Avvocato,No,Puglia,NaN,Impiegato statale,No


In [24]:
dfa.prima_classe_frequenza_scuola.value_counts()

,count
prima_classe_frequenza_scuola,
3.0,108
2.0,106
4.0,101
1.0,58
5.0,34
6.0,32
7.0,24
99.0,16
8.0,3


In [25]:
dfa['stato_provenienza'] = dfa['stato_provenienza'].str.lower()

dfa['stato_provenienza'].value_counts()

,count
stato_provenienza,
italia,97
russia,67
brasile,42
colombia,40
ucraina,40
india,25
cambogia,21
etiopia,21
other,16


In [26]:
dfa['stato_provenienza1'] = dfa['stato_provenienza1'].str.lower()

dfa['stato_provenienza1'].value_counts()

,count
stato_provenienza1,
colombia,13
russia,12
brasile,9
india,8
italia,6
cambogia,5
etiopia,4
romania,4
bolivia,4


In [27]:
region_mapping = {
    # Europe
    'italia': 'Italy',
    'russia': 'East Europe',
    'ucraina': 'East Europe',
    'ungheria': 'East Europe',
    'bulgaria': 'East Europe',
    'bielorussia': 'East Europe',
    'romania': 'East Europe',
    'polonia': 'East Europe',
    'lituania': 'East Europe',
    'moldavia': 'East Europe',
    'ukraina': 'East Europe',
    'karelia': 'East Europe',

    # Latin America
    'brasile': 'Latin America',
    'colombia': 'Latin America',
    'bolivia': 'Latin America',
    'perù': 'Latin America',
    'peru': 'Latin America',
    'peru’': 'Latin America',
    'cile': 'Latin America',
    'ecuador': 'Latin America',
    'messico': 'Latin America',
    'guatemala': 'Latin America',
    'america latina': 'Latin America',

    # Asia
    'india': 'Asia',
    'cambogia': 'Asia',
    'nepal': 'Asia',
    'vietnam': 'Asia',
    'cina': 'Asia',
    'sri lanka': 'Asia',
    'filippine': 'Asia',
    'kazakistan': 'Asia',
    'thailandia': 'Asia',
    'thailand': 'Asia',
    'korea': 'Asia',
    'cambigia': 'Asia',
    'brazile': 'Asia',

    # Africa
    'etiopia': 'Africa',
    'madagascar': 'Africa',
    'marocco': 'Africa',
    'kenya': 'Africa',
    'repubblica democratica del congo': 'Africa',
    'costa d\'avorio': 'Africa',
    'burkina faso': 'Africa',
    'mauritania': 'Africa',

    # Other/Unknown
    'other': 'Other',
}

dfa['origin_region'] = dfa['stato_provenienza'].str.lower().map(region_mapping)
dfa['origin_region'].fillna('Other', inplace=True)
dfa[['stato_provenienza', 'origin_region']].head()


,stato_provenienza,origin_region
0,italia,Italy
2,italia,Italy
3,ucraina,East Europe
4,italia,Italy
5,ucraina,East Europe


In [28]:
dfa.origin_region.value_counts()

,count
origin_region,
East Europe,151
Latin America,109
Italy,97
Asia,92
Africa,29
Other,17


In [29]:
# Define the mapping for regions based on the country names in 'stato_provenienza1'
region_mapping_1 = {
    'italia': 'Italy',
    'russia': 'East Europe',
    'brasile': 'South America',
    'colombia': 'South America',
    'ucraina': 'East Europe',
    'india': 'Asia',
    'cambogia': 'Asia',
    'etiopia': 'Africa',
    'romania': 'East Europe',
    'ungheria': 'East Europe',
    'nepal': 'Asia',
    'bolivia': 'South America',
    'perù': 'South America',
    'kazakistan': 'Asia',
    'bulgaria': 'East Europe',
    'vietnam': 'Asia',
    'polonia': 'East Europe',
    'ghana': 'Africa',
    'moldavia': 'East Europe',
    'filippine': 'Asia',
    'thailandia': 'Asia',
    'messico': 'South America',
    'costa d\'avorio': 'Africa',
    'mauritania': 'Africa',
    'cile': 'South America',
    'nazionale': 'Italy',
    'rissia': 'East Europe',
    'other': 'Other'
}

# If you have some unlisted countries, map them to 'Other' as a default fallback
dfa['origin_region1'] = dfa['stato_provenienza1'].map(region_mapping_1).fillna('No sibling')
dfa['origin_region1'].value_counts()


,count
origin_region1,
No sibling,398
South America,33
East Europe,28
Asia,22
Italy,7
Africa,7


In [30]:
from sklearn.preprocessing import LabelEncoder

frequency_map = dfa['origin_region'].value_counts().to_dict()
dfa['origin_region'] = dfa['origin_region'].map(frequency_map)

frequency_map2 = dfa['origin_region1'].value_counts().to_dict()
dfa['origin_region1'] = dfa['origin_region1'].map(frequency_map2)

# Display the transformed column
dfa['origin_region'].head()

,origin_region
0,97
2,97
3,151
4,97
5,151


In [31]:
le = LabelEncoder()
dfa[''] = le.fit_transform(dfa[''])

KeyError: ''

In [ ]:
# categorical_cols = dfa.select_dtypes(include=['object']).columns

# for col in categorical_cols:
#     if col in dfa.columns:
#         dfa[col] = dfa[col].fillna('No response')
#     else:
#         print(f"Column {col} does not exist in the DataFrame.")

##Children survey results

In [ ]:
dfc = pd.read_excel('results kids.xlsx')

In [ ]:
dfc.shape

In [ ]:
dfc = dfc.dropna(axis=1, how='all')
dfc.shape

In [ ]:
dfc.columns = [remove_italian_stopwords(col) for col in dfc.columns]

dfc.columns

In [ ]:
column_types = dfc.dtypes
type_counts = column_types.value_counts()

print("Column types and counts:")
print(type_counts)

In [ ]:
categorical_cols = dfc.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if col in dfc.columns:
        dfc[col] = dfc[col].fillna('No response')
    else:
        print(f"Column {col} does not exist in the DataFrame.")

In [ ]:
dfc.head()